In [ ]:
import os
import subprocess
from subprocess import CalledProcessError
import shlex

EXE_DIR_PATH = "../build/src/exe/"
WORK_DIR = "../ir_demo/"


def run_cmd_get_output(cmd):
    """
     limitation: cannot see progress until the process is finished. Not suitable for long-running task. 
    """
    try:       
        output = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
        output = output.decode('utf-8')
    except CalledProcessError as e:
        print(e)
        print("output: {}".format(e.output.decode('utf-8')))
    return output
        
def run_cmd(command):
    """
    get stdout realtime. 
    """
    print("$ ", command)
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if not output:
            break
        if output:
            print(output.strip().decode('utf-8'))
    rc = process.poll()
    return rc
        

cmd = "ls {EXE_DIR_PATH}".format( \
        EXE_DIR_PATH=EXE_DIR_PATH)
run_cmd(cmd)

# Image Retrieval

image retrieval system is already implemented. 

We need input for the system, and they are:

(requirements)
* project_dir
* database
* vocab_tree
* database_image_list
* query_image_list

(optional)
* num_images
* num_verifications
* max_num_features


-------------------------------------

We assume your project directory has following structure. (You can use symbolic link for images)
```
/path/to/project/...
+── images
│   +── image1.jpg
│   +── image2.jpg
│   +── ...
│   +── imageN.jpg
```

## Vocabulary

You can download pre-trained vocab tree from here: https://demuc.de/colmap/#download

Put the file into WORK_DIR


In [ ]:
DB_PATH = os.path.join(WORK_DIR, "database.db")
IMAGE_PATH = os.path.join(WORK_DIR, "images")

VOCAB_TREE_PATH = os.path.join(WORK_DIR, "vocab_tree-262144.bin")
SAVE_INDEX_PATH = os.path.join(WORK_DIR, "vocab_tree-262144_image_indexed.bin")

## Alternatively, you can build it from scratch

use `vocab_tree_builder`

In [ ]:
# TODO: run voacb_tree_builder command with appropriate args

# (Only for the first run) Create Database

To keep intermediate data for image retrieval, we use SQLite3 database. 

In [ ]:
cmd = "{EXE_DIR_PATH}database_creator --database_path {DB_PATH}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                WORK_DIR=WORK_DIR, \
                DB_PATH=DB_PATH)
run_cmd(cmd)

# Extracting Features from Images Using COLMAP

In [ ]:
cmd = "{EXE_DIR_PATH}feature_extractor --database_path {DB_PATH} --image_path {IMAGE_PATH}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                DB_PATH=DB_PATH, \
                IMAGE_PATH=IMAGE_PATH)
run_cmd(cmd)

# Timing 4 min for Oxford5k

# Import features
Most cases, you want to use pregenerated feature to do fair comparision with other models. Colmap supports feature import.

use `feature_importer`

In [ ]:
# TODO: how to import features? I want to use hessaff features. 

# Add Query Images to Database

If you want to use query images that are not in the current database (either cropped version, or new image), you have to put it in the database. 

In [ ]:
QUERY_IMAGE_PATH = "../eval/oxford5k_query_images"
QUERY_IMAGE_LIST_PATH = "../eval/oxford5k_query_image_list.txt"

cmd = "{EXE_DIR_PATH}feature_extractor --database_path {DB_PATH} --image_path {QUERY_IMAGE_PATH}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                DB_PATH=DB_PATH, \
                QUERY_IMAGE_PATH=QUERY_IMAGE_PATH)
run_cmd(cmd)

# Run Image Retrieval

`vocab_tree_retriever` contains below steps:

1. Indexing image. 
    (check this) with image feature in DB, indexing with the given vocab tree
2. 

...

If you omit query_image_list_path, it will use all images in the database. 


If database is large, making index everytime take long time. Can I save generated index?
For experiment purpose. Is it good idea to save index? Can we use the same index again, if some of algorithm is changed?


```
args:
--project_path arg
--database_path arg
--vocab_tree_path arg
--database_image_list_path arg
--query_image_list_path arg
--num_images arg (=-1)
--num_verifications arg (=0)
--max_num_features arg (=-1)

```

# Test Image Retrieval

## To get real output use below command with file redirection

```
../build/src/exe/vocab_tree_retriever --database_path ../ir_demo/database.db --vocab_tree_path ../ir_demo/vocab_tree-262144.bin --save_index_path ../ir_demo/vocab_tree-262144_image_indexed.bin --query_image_list_path ../eval/oxford5k_query_image_list.txt --num_images 1000 > query_result.txt
```


In [ ]:
NUM_VERIFICATIONS = 1000
QUERY_IMAGE_LIST_PATH = "./oxford5k_query_image_list_test.txt"
SINGLE_QUERY_IMAGE_NAME = "all_souls_000033.jpg"

with open(QUERY_IMAGE_LIST_PATH, "w") as f:
    f.write(SINGLE_QUERY_IMAGE_NAME+"\n")
    
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def parse_query_result(output):
    result = {}
    for line_idx, line in enumerate(output.split("\n")[:-1]): # -1 for trailing new line
        if line_idx == 0:
            # skip header
            continue
        if "Querying for image " in line:
            query_name = line.split(" ")[3]
            result[query_name] = []
        else:      
            result_image_filename = line.strip().split(", ")[1].split("=")[1]
            result_image_name = result_image_filename
            result[query_name].append(result_image_name)
    return result           

def show_image(image_name):
    
    image_path = os.path.join(IMAGE_PATH, image_name)
    # print("open image:", image_path)
    img_bgr = cv2.imread(image_path)

    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    plt.figure()
    plt.imshow(img)
    plt.show()
    
    

cmd = "{EXE_DIR_PATH}vocab_tree_retriever --database_path {DB_PATH} --vocab_tree_path {VOCAB_TREE_PATH} --save_index_path {SAVE_INDEX_PATH} --query_image_list_path {QUERY_IMAGE_LIST_PATH} --num_images {NUM_VERIFICATIONS}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                DB_PATH=DB_PATH, \
                VOCAB_TREE_PATH=VOCAB_TREE_PATH, \
                SAVE_INDEX_PATH=SAVE_INDEX_PATH, \
                QUERY_IMAGE_LIST_PATH=QUERY_IMAGE_LIST_PATH, \
                NUM_VERIFICATIONS=NUM_VERIFICATIONS)    
output = run_cmd_get_output(cmd)
# print(output)

result = parse_query_result(output)

for image_name, rank_list in result.items():
    print("QUERY image_name:", image_name)
    show_image(image_name)
    print("RESULT:")
    for rank, filename in enumerate(rank_list[:5]):
        print("   top{}: {}".format(rank+1, filename))
        show_image(filename)
    print()
